In [25]:
import pandas as pd
import geopandas as gpd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


In [42]:
## 12 16 18 32 35
id_prov = '35'

In [43]:
def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_GT_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data


In [44]:
df_ground_truth = pd.read_csv("/data/ksa/09_Ground Truth/gt_points.csv")
df_ground_truth = df_ground_truth.loc[df_ground_truth.iddesa.astype("str").str.startswith('32')]
df_ground_truth.head()

,iddesa,latitude,longitude,user_created_at,accuracy,fase_tumbuh,heterogenitas,topografi_sawah,observation,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,geometry,date,periode
906,3217090006,-6.165199,106.837106,2024-07-09 16:41:14,3.949533,7,1,1,7,700000mE,9300000mN,YU,48M,48MYU,POINT (106.83710634 -6.16519867),2024-07-09,2024_16
907,3210070008,-6.827025,108.239280,2024-09-18 08:39:38,1.600000,1,1,1,1,100000mE,9200000mN,AN,49M,49MAN,POINT (108.23928 -6.827025),2024-09-18,2024_22
908,3210070008,-6.827075,108.239605,2024-09-18 08:41:51,1.500000,1,1,1,1,100000mE,9200000mN,AN,49M,49MAN,POINT (108.239605 -6.827075),2024-09-18,2024_22
909,3210070008,-6.827003,108.239878,2024-09-18 08:43:11,1.500000,1,1,1,1,100000mE,9200000mN,AN,49M,49MAN,POINT (108.23987833333 -6.8270033333333),2024-09-18,2024_22
910,3210070008,-6.823270,108.234433,2024-09-18 08:48:19,1.500000,4,2,1,4,100000mE,9200000mN,AN,49M,49MAN,POINT (108.23443333333 -6.82327),2024-09-18,2024_22


In [45]:
df_ground_truth.shape

(186, 17)

In [46]:
mgrs_not_done_ = df_ground_truth.MGRS.unique()
mgrs_not_done_

array(['48MYU', '49MAN', '48MYT', '49MBN'], dtype=object)

In [47]:
df_ground_truth.groupby('MGRS').MGRS.size()

MGRS
48MYT    78
48MYU     1
49MAN    47
49MBN    60
Name: MGRS, dtype: int64

In [48]:
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_GT_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_ground_truth.loc[df_ground_truth.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    val = get_values(gc, bands_names, r['latitude'], r['longitude'])
                    df_ksa_mgrs_tmp.loc[i,bands_names] = val
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
                # break
                
            df_result = df_result[['iddesa', 'fase_tumbuh', 'heterogenitas', 
                                   'topografi_sawah', 'observation', 
                                   'latitude', 'longitude', 'MGRS','periode',
                                   'Sigma0_VH_db','Sigma0_VH_db_count']]
            with open('/data/ksa/03_Sampling/data-GT/sampling_'+id_prov+'_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

48MYU not available
49MAN available


100%|██████████| 23/23 [00:02<00:00,  8.70it/s]


done sampling for : 49MAN
48MYT available


100%|██████████| 24/24 [00:03<00:00,  6.26it/s]


done sampling for : 48MYT
49MBN available


100%|██████████| 23/23 [00:03<00:00,  7.24it/s]

done sampling for : 49MBN


In [49]:
df_result

,iddesa,fase_tumbuh,heterogenitas,topografi_sawah,observation,latitude,longitude,MGRS,periode,Sigma0_VH_db,Sigma0_VH_db_count
0,3210120001,1,1,1,1,-6.774253,108.333395,49MBN,20240101_20240112,-14.733310,2.0
1,3210120001,1,1,1,1,-6.779260,108.333768,49MBN,20240101_20240112,-14.903786,2.0
2,3210120001,1,1,1,1,-6.769170,108.321533,49MBN,20240101_20240112,-14.646858,2.0
3,3210120001,1,1,1,1,-6.774120,108.320902,49MBN,20240101_20240112,-14.594769,2.0
4,3210120001,1,1,1,1,-6.770858,108.317195,49MBN,20240101_20240112,-18.015903,2.0
...,...,...,...,...,...,...,...,...,...,...,...
55,3212150007,1,1,1,1,-6.367724,108.332187,49MBN,20240406_20240417,-16.469032,2.0
56,3212150007,2,1,1,2,-6.367273,108.332520,49MBN,20240406_20240417,-16.881393,2.0
57,3212150007,2,1,1,2,-6.367527,108.332493,49MBN,20240406_20240417,-16.831486,2.0
58,3212150005,2,1,1,2,-6.373268,108.344768,49MBN,20240406_20240417,-17.704023,2.0


### List Error : 
1. 32 : 48MYU not available
2. 12 : 48MYU not available

In [ ]:
## Cek VH Value Overtime on One Point

filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()

In [ ]:
## Open one Sampling Data

mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()

In [ ]:
df_.describe()